In [1]:
import geopandas as gpd
from shapely.geometry import Point
import netCDF4 as nc
import numpy as np
from typing import Tuple, List

ds = nc.Dataset("krill_transport_across_wap_bathy_20200101_70_pelagic_egg_drift_10.nc")

source = gpd.read_file("wap_break_shelf_epsg/wap_break_shelf_epsg_4326_wgs84_polygons.dbf")
dest = gpd.read_file("wap_chlora/wap_chlora_202001.shp")

/mnt/c/Users/bassi/Research/Connectivity/venv/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
source["source_index"] = source.index
dest["destination_index"] = dest.index

crs = source.crs
lons = ds['lon'][:]
lats = ds['lat'][:]           
firstlast = np.ma.notmasked_edges(lons, axis=1)
index_of_last = firstlast[1][1]
final_lons = lons.T[index_of_last, range(lons.T.shape[1])]
final_lats = lats.T[index_of_last, range(lons.T.shape[1])]
initial_lons = lons[:, 0]
initial_lats = lats[:, 0]
final_points = gpd.GeoDataFrame(geometry=[Point(p) for p in zip(final_lons, final_lats)], crs=crs)
initial_poins = gpd.GeoDataFrame(geometry=[Point(p) for p in zip(initial_lons, initial_lats)], crs=crs)

initial = gpd.sjoin(source, initial_poins)
initial = initial.rename(columns={"index_right": "point_id"})
final = gpd.sjoin(dest, final_points)
final = final.rename(columns={"index_right": "point_id"})

In [3]:
source_of_each_point = initial.loc[:, ["source_index", "point_id"]]
supply_of_each_source = (
    source_of_each_point.groupby("source_index")
    .count()
    .rename(columns={"point_id": "supplied_amount"})
    .merge(source_of_each_point, on="source_index")
)

destination_of_each_point = final.loc[:, ["destination_index", "point_id"]]

settle_and_supply_of_each_sector = (
    supply_of_each_source.merge(destination_of_each_point, on="point_id")
    .groupby(["source_index", "supplied_amount", "destination_index"])
    .count()
    .rename(columns={"point_id": "settled_amount"})
    .reset_index()
)
settle_and_supply_of_each_sector["connectivity"]= (
    settle_and_supply_of_each_sector.apply(lambda x: x["settled_amount"] / x["supplied_amount"], axis=1)
)

In [4]:
settle_and_supply_of_each_sector

,source_index,supplied_amount,destination_index,settled_amount,connectivity
0,0,271,3,17,0.062731
1,0,271,12,3,0.011070
2,45,3572,95,1,0.000280
3,45,3572,112,12,0.003359
4,45,3572,113,6,0.001680
5,45,3572,114,1,0.000280
6,45,3572,115,2,0.000560
7,45,3572,116,7,0.001960
8,45,3572,122,1,0.000280
9,45,3572,123,5,0.001400
